In [ ]:
pip install --upgrade pip

Our libraries for the project

In [ ]:
!pip install torch torchvision open3d matplotlib tqdm pillow


Now importing them to use it

In [ ]:
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
import open3d as o3d
from tqdm import tqdm


In [ ]:
!pip install timm


Loading the Midas model

In [ ]:
print("Loading Midas model my user")
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Large")
midas.eval().to("cuda" if torch.cuda.is_available() else "cpu")
transform = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
print("✅ MiDaS loaded now you can go to other cell")


#sometimes it will show error then delete the previos loaded model ,then run this code again


In [ ]:
def estimate_depth_and_create_point_cloud(img: Image.Image):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_tensor = transform(np.array(img)).to(device)

    with torch.no_grad():
        prediction = midas(input_tensor)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.size[::-1],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()

    # Show depth map
    plt.figure(figsize=(8, 6))
    plt.imshow(depth_map, cmap='inferno')
    plt.title("Depth Map")
    plt.axis('off')
    plt.show()

    # Convert depth to point cloud (XYZ)
    def depth_to_point_cloud(depth, image):
        h, w = depth.shape
        fx = fy = 1.0
        cx, cy = w / 2, h / 2

        points, colors = [], []
        image = np.array(image)

        for y in tqdm(range(h)):
            for x in range(w):
                Z = depth[y, x]
                X = (x - cx) * Z / fx
                Y = (y - cy) * Z / fy
                points.append([X, Y, Z])
                colors.append(image[y, x] / 255.0)

        return np.array(points), np.array(colors)

    return depth_map, *depth_to_point_cloud(depth_map, img)


In [ ]:
from IPython.display import display, FileLink

def create_and_save_mesh(points, colors, obj_filename="output_model.obj", stl_filename="output_model.stl"):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    pcd.estimate_normals()
    print("Starting mesh creation...")
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=7)
    print("Mesh created.")


    mesh.compute_vertex_normals()
    o3d.io.write_triangle_mesh(obj_filename, mesh)
    o3d.io.write_triangle_mesh(stl_filename, mesh)
    print(f"✅ 3D model saved as '{obj_filename}' and '{stl_filename}'")

    # Display download links in the notebook
    display(FileLink(obj_filename))
    display(FileLink(stl_filename))

    return mesh


In [ ]:
!pip install ipywidgets


In [ ]:
import ipywidgets as widgets


In [ ]:
from IPython.display import clear_output


In [ ]:
def visualize_mesh_headless(mesh, width=640, height=480):
    print("Opening 3D mesh in a separate window...")
    o3d.visualization.draw_geometries([mesh])


In [ ]:
import io

output_widget = widgets.Output()

upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
generate_button = widgets.Button(description="Generate 3D Model", button_style='success')

def on_generate_clicked(b):
    with output_widget:
        clear_output()
        if not upload_widget.value:
            print("❌ Please upload an image first.")
            return

        for file_info in upload_widget.value:
            img = Image.open(io.BytesIO(file_info['content'])).convert("RGB")
            print(f"✅ Uploaded: {file_info['name']}")
            display(img.resize((200, 200)))

            depth_map, points, colors = estimate_depth_and_create_point_cloud(img)
            mesh = create_and_save_mesh(points, colors)
            print("🟢 3D mesh preview (static image):")
            visualize_mesh_headless(mesh)

display(widgets.VBox([
    widgets.Label("📷 Upload an Image:"),
    upload_widget,
    generate_button,
    output_widget
]))

generate_button.on_click(on_generate_clicked)
